In [32]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm  # For progress bars


In [40]:
# Step 2: Define Transformations
# Explanation: These transformations will prepare the images for the model by resizing, normalizing, and augmenting.
transform = transforms.Compose([
    transforms.Resize((128, 128)),        # Resize images to 128x128 pixels
    transforms.RandomHorizontalFlip(),    # Randomly flip images horizontally to augment data
    transforms.ToTensor(),                # Convert images to PyTorch tensors
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize pixel values to [-1, 1]
])


In [44]:
# Step 3: Load Dataset and Split into Train and Validation Sets

# Explanation: Loading images from the directory and splitting the dataset into 80% training and 20% validation.

dataset = datasets.ImageFolder(root='//Users/muhammadhassanzahoor/Desktop/EAI 6010 - Applications of Artificial Intelligence/Module 2/archive', transform=transform)  # Load dataset from directory
train_size = int(0.8 * len(dataset))  # Calculate 80% of dataset size for training
val_size = len(dataset) - train_size  # Remaining 20% for validation
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])  # Split dataset


In [46]:
# Step 4: Create DataLoaders for Batch Processing

# Explanation: DataLoaders help load data in batches, making training more efficient.

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # DataLoader for training data
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)  # DataLoader for validation data


In [48]:
# Step 5: Load Pre-Trained Model and Modify for Binary Classification

# Explanation: We use a pre-trained ResNet18 model and replace the final layer for our specific task (cats vs. dogs).

model = models.resnet18(pretrained=True)  # Load pre-trained ResNet18 model
for param in model.parameters():
    param.requires_grad = False  # Freeze all pre-trained layers to keep their learned weights
num_features = model.fc.in_features  # Get the number of input features to the final layer
model.fc = nn.Linear(num_features, 2)  # Replace final layer with a binary classifier (2 classes: cat and dog)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Use GPU if available
model = model.to(device)  # Move model to the device


In [50]:
# Step 6: Define Loss Function and Optimizer

# Explanation: We set up the loss function and optimizer to guide how the model learns.

criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification tasks
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)  # Adam optimizer, only updating the final layer


In [56]:
# Step 7: Training Loop
# Explanation: The training loop goes through the data multiple times (epochs) to learn the classification task.
epochs = 5  # Number of epochs to train the model
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0  # Track the loss for each epoch
    
    for images, labels in tqdm(train_loader):  # Loop over each batch
        images, labels = images.to(device), labels.to(device)  # Move data to the device
        
        optimizer.zero_grad()  # Clear previous gradients
        outputs = model(images)  # Forward pass: get model predictions
        loss = criterion(outputs, labels)  # Calculate the loss
        loss.backward()  # Backward pass: compute gradients
        optimizer.step()  # Update weights based on gradients
        
        running_loss += loss.item()  # Accumulate loss

    # Print the average loss for this epoch
    print(f"Epoch {epoch+1}/{epochs}, Training Loss: {running_loss/len(train_loader)}")


100%|███████████████████████████████████████████| 25/25 [00:09<00:00,  2.52it/s]


Epoch 1/10, Training Loss: 0.2158990502357483


100%|███████████████████████████████████████████| 25/25 [00:09<00:00,  2.53it/s]


Epoch 2/10, Training Loss: 0.2156403636932373


100%|███████████████████████████████████████████| 25/25 [00:09<00:00,  2.56it/s]


...

Epoch 8/10, Training Loss: 0.1762071368098259


100%|███████████████████████████████████████████| 25/25 [00:10<00:00,  2.48it/s]


Epoch 9/10, Training Loss: 0.15259778171777724


100%|███████████████████████████████████████████| 25/25 [00:10<00:00,  2.46it/s]

Epoch 10/10, Training Loss: 0.17348867893218994


In [58]:
# Step 8: Validation Loop
# Explanation: After each epoch, we evaluate the model on the validation set to monitor for overfitting.
model.eval()  # Set model to evaluation mode (no gradient updates)
val_loss = 0.0
correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculation for validation to save memory
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to device
        
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Calculate validation loss
        val_loss += loss.item()  # Accumulate validation loss
        
        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)  # Get the predicted class
        total += labels.size(0)  # Total number of samples
        correct += (predicted == labels).sum().item()  # Count correct predictions

# Print validation loss and accuracy after each epoch
print(f"Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%")


Validation Loss: 0.1562096859727587, Accuracy: 94.0%
